In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
import pandas as pd

import os
import sys

# Get the current directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)
sys.path.append(os.path.join(parent_dir, 'turning_scripts'))

from data_preambles import dtypes_no_id, col_names, csv_to_exclude, catalog_col_names
from path_prefix import PATH_PREFIX



In [ ]:
# Load one CSV file
hour_df = pd.read_csv(f'{PATH_PREFIX}/data/csv/1716508800.csv', dtype=dtypes_no_id, parse_dates=True)
hour_df.columns = col_names
hour_df.head(5)
# Add an id column
hour_df['id'] = hour_df['icao24'] + hour_df['callsign']
hour_df.head(5)


In [ ]:
hour_ids = hour_df['id'].unique()
print(f'There are {len(hour_ids)} unique ids in the hour_df')

In [6]:
from get_turns import get_turning_points, TurnAndRise
from collections import deque
from tqdm import tqdm
from cleaning_script import clean_by_speed as cleaner

# Note that get_turning_points will automatically add the first and last point to the list of turning points
# So we don't need to add them manually

In [ ]:
# For each callsign, we attempt to get the turning points
print('Creating segments...')
print('Caution: will skip N/A rows')

seg_from_lat = deque()
seg_from_lon = deque()
seg_to_lat = deque()
seg_to_lon = deque()

callsigns_skipped = 0
for id in tqdm(hour_ids):
    try:
        df_id = hour_df[hour_df['id'] == id]
        # Clean the dataframe
        df_id = cleaner.clean_trajectory(df_id)
        tr = get_turning_points(df_id)
    except ValueError as e:
        # print(f'Skipping {id} because {e}')
        callsigns_skipped += 1
        continue

    # For each turn, we get the segment from and to
    for i in range(len(tr['tp_time']) - 1):
        seg_from_lat.append(tr['tp_lat'][i])
        seg_from_lon.append(tr['tp_lon'][i])
        seg_to_lat.append(tr['tp_lat'][i+1])
        seg_to_lon.append(tr['tp_lon'][i+1])

print(f'There were {len(hour_ids)} callsigns, of which {callsigns_skipped} were skipped')
    

In [7]:
from datetime import datetime, timezone

timestamp = 1716508800

# Convert timestamp to datetime object in UTC
utc_datetime = datetime.fromtimestamp(timestamp, tz=timezone.utc)

# Format the datetime as a string
formatted_datetime = utc_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

In [ ]:
# Plot all the segments on a map

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Create a new figure and axis with a map projection
fig, ax = plt.subplots(figsize=(12, 10), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_aspect('auto')  # This allows the aspect ratio to adjust naturally

# Add map features
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Set the extent of the map to cover Europe
ax.set_extent([-10, 30, 35, 60], crs=ccrs.PlateCarree())

# Plot the segments
for from_lat, from_lon, to_lat, to_lon in zip(seg_from_lat, seg_from_lon, seg_to_lat, seg_to_lon):
    ax.plot([from_lon, to_lon], [from_lat, to_lat], 
            color='red', linewidth=0.5, alpha=0.5, 
            transform=ccrs.Geodetic())

# Add gridlines
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Add a title
plt.title(f'Flight Segments over Europe, {formatted_datetime}')

# Show the plot
plt.show()
# Create a directory for the plots if it doesn't exist
# os.makedirs(f'{PATH_PREFIX}/data/plots', exist_ok=True)
# plt.savefig(f'{PATH_PREFIX}/data/plots/flight_segments_{timestamp}.png')

In [ ]:
# Save the segments to a pickle file
import pickle

# Create a dictionary to store the segment data
segment_data = {
    'seg_from_lat': seg_from_lat,
    'seg_from_lon': seg_from_lon,
    'seg_to_lat': seg_to_lat,
    'seg_to_lon': seg_to_lon
}

# Create a directory for the pickle files if it doesn't exist
os.makedirs(f'{PATH_PREFIX}/data/segments', exist_ok=True)

# Save the segment data to a pickle file
with open(f'{PATH_PREFIX}/data/segments/flight_segments_{timestamp}.segments.pickle', 'wb') as f:
    pickle.dump(segment_data, f)

print(f"Segment data saved to {PATH_PREFIX}/data/segments/flight_segments_{timestamp}.segments.pickle")
